In [1]:
%run ipy_setup.py
%matplotlib inline

In [4]:
import pysam
aln_bamfile = pysam.AlignmentFile('/hpc/users/neffr01/jason_new/falcon_unzip_jason_071715/3-falcon_unzip/000000F/aln_sorted.bam', 'rb')

In [7]:
out_fp = pysam.AlignmentFile('/hpc/users/neffr01/jason_new/falcon_unzip_jason_071715/3-falcon_unzip/000000F/aln_sorted.ann.bam', 'wb', template=aln_bamfile)

In [10]:
hap_info = pd.read_csv('/hpc/users/neffr01/jason_new/falcon_unzip_jason_071715/3-falcon_unzip/000000F/phased_reads', sep=' ', header=None)

In [11]:
hap_info

,0,1,2,3,4,5
0,0,1,1,0,7,m141211_104910_42177R_c10076198255000000182316...
1,1,1,0,3,0,m141211_104910_42177R_c10076198255000000182316...
2,2,1,1,0,12,m150202_190108_42163R_c10077989255000000182316...
3,3,1,0,8,0,m150211_010455_42163R_c10078001255000000182316...
4,4,1,1,0,6,m150211_072408_42163R_c10078001255000000182316...
5,5,1,1,0,5,m150214_045541_42177R_c10077999255000000182316...
6,6,1,0,15,0,m150122_080841_42156_c100779752550000001823165...
7,7,1,1,0,3,m150221_193226_42177R_c10078196255000000182315...
8,8,1,1,0,13,m150104_114301_42177R_c10076178255000000182316...
9,9,1,0,3,0,m150131_033204_42156_c100779802550000001823165...


In [12]:
phased_set = frozenset(hap_info[5])

In [14]:
hap_info.set_index(hap_info[5], inplace=True)

In [25]:
import pysam
aln_bamfile = pysam.AlignmentFile('/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_with_indels/hg002_000000F.new.merged.ann-indels.bam', 'rb')
out_fp = pysam.AlignmentFile('/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_with_indels/hg002_000000F.new.merged.ann-indels.ann-falcon.bam', 'wb', template=aln_bamfile)

count = 0
for bamread in aln_bamfile.fetch():
    count += 1
    if (count % 100) == 0:
        sys.stdout.write("\rRead %s lines from input." % str(count))
        sys.stdout.flush()
    if bamread.query_name in phased_set:
        readline = hap_info.ix[bamread.query_name]
        hapblock = str(readline[1])
        haplotype = str(readline[2])
        bamread.tags += [("ZF", hapblock + "," + haplotype)] # add info
        out_fp.write(bamread)
    else:
        out_fp.write(bamread)

out_fp.close()
aln_bamfile.close()

Read 290800 lines from input.